In [14]:
import uproot
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import time
%matplotlib inline
import ROOT
%jsroot on

In [22]:
cell_map_CE = np.load("files/reduced_transverse_granularity_by16_cell_map_CEE_CEH.npy")
cell_xy_CE = np.load("files/reduced_transverse_granularity_by16_cell_xy_CEE_CEH.npy")

In [23]:
cell_map_AHCAL = np.load("files/reduced_transverse_granularity_by16_cell_map_AHCAL.npy")
cell_xy_AHCAL = np.load("files/reduced_transverse_granularity_by16_cell_xy_AHCAL.npy")

In [4]:
#cell_map_CE = np.load("files/reduced_transverse_granularity_cell_map_CEE_CEH.npy")
#cell_xy_CE = np.load("files/reduced_transverse_granularity_cell_xy_CEE_CEH.npy")

In [5]:
#cell_map_AHCAL = np.load("files/reduced_transverse_granularity_cell_map_AHCAL.npy")
#cell_xy_AHCAL = np.load("files/reduced_transverse_granularity_cell_xy_AHCAL.npy")

In [24]:
def new_rechits_ce(df_):
    
    cell_idxs = []
    rechit_x = []
    rechit_y = []
    rechit_z = []
    rechit_ene = []
    rechit_lay = []
    
    rec_lay = df_.rechit_layer
    
    for lay in range(1,41):
    
        rec_x = df_.rechit_x[rec_lay==lay]
        rec_y = df_.rechit_y[rec_lay==lay]
        rec_z = df_.rechit_z[rec_lay==lay]
        rec_ene = df_.rechit_energy[rec_lay==lay]
        
        new_cell_idxs = []
        new_rechit_x = []
        new_rechit_y = []
        new_rechit_z = []
        new_rechit_ene = []
        new_rechit_lay = []

        for rec_i in range(rec_x.size):


            xy_array = np.array(np.array([rec_x[rec_i], rec_y[rec_i]]).tolist()).round(5)
            #print(xy_array)


            cell_idx = cell_map_CE[:,2][(cell_map_CE[:,0:2]==xy_array).all(axis=1)][0]
            #print(cell_idx,"\n")


            if cell_idx not in new_cell_idxs:

                new_cell_idxs.append(cell_idx)

                new_rechit_x.append(cell_xy_CE[cell_xy_CE[:,2]==cell_idx][0][0])
                new_rechit_y.append(cell_xy_CE[cell_xy_CE[:,2]==cell_idx][0][1])
                new_rechit_z.append(rec_z[rec_i])
                new_rechit_ene.append(rec_ene[rec_i])
                new_rechit_lay.append(lay)

            else: new_rechit_ene[new_cell_idxs.index(cell_idx)] += rec_ene[rec_i]
                
        cell_idxs += new_cell_idxs
        rechit_x += new_rechit_x
        rechit_y += new_rechit_y
        rechit_z += new_rechit_z
        rechit_ene += new_rechit_ene
        rechit_lay += new_rechit_lay
        
            
    
    
    return [np.array(rechit_x), np.array(rechit_y), np.array(rechit_z), np.array(rechit_ene), np.array(rechit_lay)]
            
              

In [25]:
trim_ahcal_lay = np.array([ 3,  7, 11, 15, 19, 23, 27, 31, 35, 39])

def new_rechits_ahc(df_):
    
    cell_idxs = []
    ahc_hit_x = []
    ahc_hit_y = []
    ahc_hit_z = []
    ahc_hit_ene = []
    ahc_hit_lay = []
    
    rec_lay = df_.ahc_hitK
    
    for lay in range(1,40):
        
        if lay not in trim_ahcal_lay: continue
    
        rec_x = df_.ahc_hitX[rec_lay==lay]
        rec_y = df_.ahc_hitY[rec_lay==lay]
        rec_z = df_.ahc_hitZ[rec_lay==lay]
        rec_ene = df_.ahc_hitEnergy[rec_lay==lay]
        
        new_cell_idxs = []
        new_ahc_hit_x = []
        new_ahc_hit_y = []
        new_ahc_hit_z = []
        new_ahc_hit_ene = []
        new_ahc_hit_lay = []

        for rec_i in range(rec_x.size):


            xy_array = np.array(np.array([rec_x[rec_i], rec_y[rec_i]]).tolist()).round(1)
            #print(xy_array)


            cell_idx = cell_map_AHCAL[:,2][(cell_map_AHCAL[:,0:2]==xy_array).all(axis=1)][0]
            #print(cell_idx,"\n")


            if cell_idx not in new_cell_idxs:

                new_cell_idxs.append(cell_idx)

                new_ahc_hit_x.append(cell_xy_AHCAL[cell_xy_AHCAL[:,2]==cell_idx][0][0])
                new_ahc_hit_y.append(cell_xy_AHCAL[cell_xy_AHCAL[:,2]==cell_idx][0][1])
                new_ahc_hit_z.append(rec_z[rec_i])
                new_ahc_hit_ene.append(rec_ene[rec_i])
                new_ahc_hit_lay.append(lay)

            else: new_ahc_hit_ene[new_cell_idxs.index(cell_idx)] += rec_ene[rec_i]
                
        cell_idxs += new_cell_idxs
        ahc_hit_x += new_ahc_hit_x
        ahc_hit_y += new_ahc_hit_y
        ahc_hit_z += new_ahc_hit_z
        ahc_hit_ene += new_ahc_hit_ene
        ahc_hit_lay += new_ahc_hit_lay
            
    
    
    return [np.array(ahc_hit_x), np.array(ahc_hit_y), np.array(ahc_hit_z)+159.40000049999998, np.array(ahc_hit_ene), np.array(ahc_hit_lay)]

In [26]:
def comb_x(df_):
    return np.concatenate([df_.new_rechit_ce_x, df_.new_rechit_ahc_x], axis=0)

def comb_y(df_):
    return np.concatenate([df_.new_rechit_ce_y, df_.new_rechit_ahc_y], axis=0)
    
def comb_z(df_):
    return np.concatenate([df_.new_rechit_ce_z, df_.new_rechit_ahc_z], axis=0)

def comb_ene(df_):
    return np.concatenate([df_.new_rechit_ce_ene, df_.new_rechit_ahc_ene], axis=0)

In [27]:
data = uproot.open("/eos/user/n/nkasarag/NSM_5M_TB_HGCAL_flat_pion_samples/skimmed_ntuple_sim_config22_pions_1to2M_combinedHgc_Ahc_v46.root")

lst = ['rechit_x',
 'rechit_y',
 'rechit_z',
      'rechit_layer',
      'rechit_energy',
      'ahc_hitK',
 'ahc_hitEnergy',
 'ahc_hitX',
 'ahc_hitY',
 'ahc_hitZ',
      'trueBeamEnergy']

In [28]:
num_events = data["pion_variables_v1"].pandas.df("trueBeamEnergy", flatten=False).trueBeamEnergy.size
print(num_events)


#event_lst = [0, 100000, 200000, 300000, 400000, num_events]
event_lst = [0, 100000, 200000, 300000, 400000, 500000, 600000, 700000, num_events]
event_lst

837795


[0, 100000, 200000, 300000, 400000, 500000, 600000, 700000, 837795]

In [29]:
for i in range(9):
    
    time1 = time.time()
    
    df = data["pion_variables_v1"].pandas.df(lst, flatten=False, entrystart=event_lst[i], entrystop=event_lst[i+1])
    df.reset_index(inplace=True)
    
    
    rechits_ce = df.apply(new_rechits_ce, axis=1)
    ce_df = pd.DataFrame(rechits_ce.values.tolist(), columns=["new_rechit_ce_x", "new_rechit_ce_y", "new_rechit_ce_z", "new_rechit_ce_ene", "new_rechit_ce_lay"])

    rechits_ahc = df.apply(new_rechits_ahc, axis=1)
    ahc_df = pd.DataFrame(rechits_ahc.values.tolist(), columns=["new_rechit_ahc_x", "new_rechit_ahc_y", "new_rechit_ahc_z", "new_rechit_ahc_ene", "new_rechit_ahc_lay"])
    
    df = pd.concat([df, ce_df, ahc_df], axis=1)
    
    
    df["reduced_x"] = df.apply(comb_x, axis=1)
    df["reduced_y"] = df.apply(comb_y, axis=1)
    df["reduced_z"] = df.apply(comb_z, axis=1)
    df["reduced_ene"] = df.apply(comb_ene, axis=1)
    
    
    df_to_save = df[["trueBeamEnergy", "reduced_x", "reduced_y", "reduced_z", "reduced_ene"]]
    
    
    df_to_save.to_pickle("pkl_files/red_by16/1to2M/event_%sto%s_0to1M.pickle"%(str(event_lst[i]), str(event_lst[i+1])))
    
    print(i+1, "-> mins = ", (time.time()-time1)/60)


1 -> mins =  90.63150891860326
2 -> mins =  95.36786881685256
3 -> mins =  91.01108336051306
4 -> mins =  85.39121867020926
5 -> mins =  86.34721855719884
6 -> mins =  86.01442895730337


KeyboardInterrupt: 

In [14]:
import pickle

with open("pkl_files/test.pickle", "rb") as f:
    a=pickle.load(f)

In [15]:
a.head()

,trueBeamEnergy,reduced_x,reduced_y,reduced_z,reduced_ene
0,232.958725,"[-1.45764, -1.45764, -1.45764, -1.45764, -1.45...","[2.52472, 2.52472, 2.52472, 2.52472, 2.52472, ...","[13.8775, 14.7675, 16.7825, 17.6725, 19.6875, ...","[24.004648, 47.36815, 79.67454, 328.24393, 391..."
1,59.094654,"[-3.40117, -3.40117, -3.40117, -3.40117, -3.40...","[2.52472, 2.52472, 2.52472, 2.52472, 2.52472, ...","[13.8774995803833, 14.767499923706055, 16.7824...","[20.518417358398438, 49.82566833496094, 27.543..."
2,250.189209,"[-3.40117, -3.40117, -3.40117, -3.40117, -3.40...","[2.52472, 2.52472, 2.52472, 2.52472, 2.52472, ...","[13.8775, 14.7675, 16.7825, 17.6725, 19.6875, ...","[48.838333, 71.61738, 54.37345, 104.23062, 218..."
3,312.932007,"[-1.45764, -1.45764, -1.45764, -1.45764, -1.45...","[2.52472, 2.52472, 2.52472, 2.52472, 2.52472, ...","[13.8775, 14.7675, 16.7825, 17.6725, 19.6875, ...","[61.70473, 103.26814, 82.29268, 49.790646, 186..."
4,62.493023,"[-3.40117, -3.40117, -3.40117, -3.40117, -3.40...","[2.52472, 2.52472, 2.52472, 0.28053, 2.52472, ...","[13.8775, 14.7675, 16.7825, 16.7825, 17.6725, ...","[32.033787, 63.76382, 119.42364, 63.976162, 59..."


In [16]:
np.unique(np.hstack(a.reduced_z.values))

array([ 13.87749958,  14.76749992,  16.78249931,  17.67250061,
        19.6875    ,  20.57749939,  22.69249916,  23.58250046,
        25.69750023,  26.58749962,  28.70249939,  29.59250069,
        31.50749969,  32.39749908,  34.3125    ,  35.20249939,
        37.11750031,  38.00749969,  39.92250061,  40.8125    ,
        42.90750122,  44.03749847,  46.41249847,  47.54249954,
        49.68199921,  50.68799973,  52.88150024,  53.9034996 ,
        64.61849976,  71.3259964 ,  78.03350067,  84.84100342,
        91.54850006,  98.23600006, 115.15299988, 122.56099701,
       129.96899414, 137.79600525, 145.20300293, 152.51100159,
       166.91999817, 177.3999939 , 187.87998962, 198.35998535,
       208.83999634, 219.31999207, 229.79998779, 240.27999878,
       250.75999451, 261.23999023])

In [1]:
np.unique(np.hstack(a.reduced_z.values))

NameError: name 'np' is not defined